# 0. Imports

In [ ]:
import os
os.chdir('./dash_interface/')

In [ ]:
from my_dash_components import ScatterplotComponent
from dash import dcc, html, Input, Output
import pandas as pd
from jupyter_dash import JupyterDash

In [ ]:
hostId = 0
def getHost():
    global hostId
    hostId += 1
    return '127.0.0.' + str(hostId)

# 1. Gráfico customizado com widget do dash 

In [ ]:
filename =  '../web3/data.tsv'
iris = pd.read_csv(filename,sep="\t")
iris_array_of_dicts = iris.to_dict(orient='records')

app = JupyterDash(__name__)

app.layout = html.Div([
    ScatterplotComponent(
        id='input',
        data=iris_array_of_dicts
    ),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=2.5, step=0.1,
        marks={0: '0', 2.5: '2.5'},
        value=[0.5, 2]
    ),
])

@app.callback(
    Output("input", "data"), 
    Input("range-slider", "value"))
def update_bar_chart(value):
    low, high = value
    mask = (iris['petalWidth'] > low) & (iris['petalWidth'] < high)
    res = iris[mask].to_dict(orient='records')
    return res

if __name__ == '__main__':
    app.run_server(mode="jupyterlab", host=getHost())
